In [13]:
import torch
import torchvision
from torch.utils.data import DataLoader 
from torch import nn
from torch.nn import Conv2d,Linear,MaxPool2d,Flatten
from torch.utils.tensorboard import SummaryWriter
import time
torch.cuda.is_available()


True

In [14]:
train_data = torchvision.datasets.CIFAR10("./CIFAR10",train=True,download = True,transform= torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10("./CIFAR10",train=False,download = True,transform= torchvision.transforms.ToTensor())
train_data_size= len(train_data)
test_data_size= len(test_data)
print("训练集的大小为:{}".format(train_data_size))
print("测试集的大小为:{}".format(test_data_size))
writer = SummaryWriter("logs")

Files already downloaded and verified
Files already downloaded and verified
训练集的大小为:50000
测试集的大小为:10000


In [15]:
train_dataloader=DataLoader(train_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)


In [16]:
class Miles(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1= Conv2d(3,32,5,padding=2)
        # self.maxpool1=MaxPool2d(2)
        # self.conv2=Conv2d(32,32,5,padding=2)
        # self.maxpool2=MaxPool2d(2)
        # self.conv3=Conv2d(32,64,5,padding=2)
        # self.maxpool3=MaxPool2d(2)
        # self.flatten=Flatten()
        # self.linear1=Linear(1024,64)
        # self.linear2=Linear(64,10)
        self.model=nn.Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10))
    def forward(self,x):
        # x=self.conv1(x)
        # x=self.maxpool1(x)
        # x=self.conv2(x)
        # x=self.maxpool2(x)
        # x=self.conv3(x)
        # x=self.maxpool3(x)
        # x=self.flatten(x)
        # x=self.linear1(x)
        # x=self.linear2(x)
        x=self.model(x)
        return x

miles = Miles()
if torch.cuda.is_available():
    miles.cuda()
loss_fn =  nn.CrossEntropyLoss()
loss_fn.cuda()
learning_rate = 1e-2
optimizer = torch.optim.SGD(miles.parameters(),lr=learning_rate)
total_train_step = 0
total_test_step = 0 
epoch=10

In [17]:
for i in range (epoch):
    print("----------第{}轮训练开始----------".format(i+1))

    #train
    miles.train()
    for data in train_dataloader:
        imgs,targets = data
        imgs=imgs.cuda()
        targets = targets.cuda()
        outputs = miles(imgs)
        loss=loss_fn(outputs,targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if total_train_step%100==0:
            print("训练次数:{},Loss:{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),total_train_step)

    #test
    miles.eval()
    total_accuracy = 0
    loss = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets =data
            imgs=imgs.cuda()
            targets = targets.cuda()
            outputs = miles(imgs)
            loss+=loss_fn(outputs,targets)
            accuray=(outputs.argmax(1)==targets).sum()
            total_accuracy+=accuray
    total_test_step+=1
    print("整体测试集上的loss:{}".format(loss))
    print("整体测试集上的accuracy:{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss",loss.item(),total_test_step)
    writer.add_scalar("accuracy",total_accuracy/test_data_size,total_test_step)


----------第1轮训练开始----------
训练次数:100,Loss:2.2903764247894287
训练次数:200,Loss:2.283857583999634
训练次数:300,Loss:2.271825075149536
训练次数:400,Loss:2.194459915161133
训练次数:500,Loss:2.0597660541534424
训练次数:600,Loss:2.0331740379333496
训练次数:700,Loss:1.9803344011306763
整体测试集上的loss:310.6605529785156
整体测试集上的accuracy:0.28619998693466187
----------第2轮训练开始----------
训练次数:800,Loss:1.8498244285583496
训练次数:900,Loss:1.7939730882644653
训练次数:1000,Loss:1.8985542058944702
训练次数:1100,Loss:1.9946337938308716
训练次数:1200,Loss:1.6889396905899048
训练次数:1300,Loss:1.647009015083313
训练次数:1400,Loss:1.731516718864441
训练次数:1500,Loss:1.7749844789505005
整体测试集上的loss:300.6021423339844
整体测试集上的accuracy:0.3165999948978424
----------第3轮训练开始----------
训练次数:1600,Loss:1.7246458530426025
训练次数:1700,Loss:1.6456791162490845
训练次数:1800,Loss:1.9350626468658447
训练次数:1900,Loss:1.7327171564102173
训练次数:2000,Loss:1.9106097221374512
训练次数:2100,Loss:1.525888204574585
训练次数:2200,Loss:1.4915214776992798
训练次数:2300,Loss:1.779623031616211
整体测试集上的loss:263.733